# Введение в машинное обучение

Заказчиком представлена база данных об использовании клиентами услуг связи. Отражена информация по клиентам, обслуживающимся в рамках актуальных тарифных планов (smart и ultra). 
Поставлена задача на основании представленных сведений, подготовить модель, предлагающую такой актуальный тарифный план (smart либо ultra) клиенту, находящемуся на обслуживании в рамках одного из устаревших тарифных планов.
Установлено требование о достижении моделью значений accuracy не менее 0.75.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных-из-файла" data-toc-modified-id="Изучение-данных-из-файла-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных из файла</a></span>
    <ul class="toc-item"><li><span><a href="#Вывод-по-анализу-данных" data-toc-modified-id="Вывод по анализу данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод по анализу данных</a></span></li></ul></li>
    <li><span><a href="#Формирование-выборок" data-toc-modified-id="Формирование-выборок-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Формирование выборок</a></span></li>
    <li><span><a href="#Исследование-качества-моделей" data-toc-modified-id="Исследование-качества-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование качества моделей</a></span></li>
    <ul class="toc-item"><li><span><a href="#Вывод-по-исследованию-качества-моделей" data-toc-modified-id="Вывод-по-исследованию-качества-моделей-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод по исследованию качества моделей</a></span></li></ul>
    <li><span><a href="#Настройка-гиперпараметров-моделей" data-toc-modified-id="Настройка-гиперпараметров-моделей-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Настройка гиперпараметров моделей</a></span></li>
    <li><span><a href="#Финальное-тестирование-и-вывод" data-toc-modified-id="Финальное-тестирование-и-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Финальное тестирование и вывод</a></span></li>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Изучение данных из файла

In [2]:
try:
    data = pd.read_csv('/Users/gorbachevalexander/Desktop/Машинка/users_behavior.csv', sep=',')
except:
    data = pd.read_csv('/datasets/users_behavior.csv', sep=',')

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
data[data.sum(axis=1)==0]['calls'].count()

0

 ### Вывод по анализу данных

Представлены сведения об использовании услуг связи за месяц по неименованным пользователям (привязка сведений к ID клиента отсутствует).
Всего 3214 записей. 30,6% записей (.mean() по полю "is_ultra") приходится на пользователей тарифа "Ультра". 
Записи о клиентах, не пользовавшихся в течение месяца услугами отсутствуют.
Согласно describe() заведомо некорректные значения (выбросы) отсутствуют.
Типы данных соответствуют информации и предполагаемым итерациям в рамках проекта.
Предобработка в части заполнения пропусков, изменения типов данных, выбросов и заведомо некорректных значений не требуется.

# Формирование выборок

In [7]:
base_data = data.drop('is_ultra',axis=1)
base_target=data['is_ultra']

In [8]:
train_data,test_data,train_target,test_target = train_test_split(base_data,base_target,test_size=0.2,random_state=12345)

In [9]:
train_data,valid_data,train_target,valid_target = train_test_split(train_data,train_target, test_size=0.25,random_state=12345)

# Исследование качества моделей

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

In [11]:
model_regre = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_regre.fit(train_data,train_target)
accuracy_regre =  model_regre.score(valid_data,valid_target)
print('Оценка на валидации', accuracy_regre)
overeducate_regre_base = model_regre.score(train_data,train_target)
print('Оценка на учебной выборке', overeducate_regre_base)

Оценка на валидации 0.7262830482115086
Оценка на учебной выборке 0.7510373443983402


In [12]:
model_tree = DecisionTreeClassifier(random_state=12345)
model_tree.fit(train_data,train_target)
accuracy_tree = model_tree.score(valid_data,valid_target)
print('Оценка на валидации', accuracy_tree)
overeducate_tree_base = model_tree.score(train_data,train_target)
print('Оценка на учебной выборке', model_tree.score(train_data,train_target))

Оценка на валидации 0.7122861586314152
Оценка на учебной выборке 1.0


In [13]:
model_forest = RandomForestClassifier(random_state=12345)
model_forest.fit(train_data,train_target)
accuracy_forest = model_forest.score(valid_data,valid_target)
print('Оценка на валидации', accuracy_forest)
overeducate_forest_base = model_forest.score(train_data,train_target)
print('Оценка на учебной выборке', model_forest.score(train_data,train_target))

Оценка на валидации 0.7947122861586314
Оценка на учебной выборке 1.0


### Вывод по исследованию качества моделей
1. Результат обучения основных моделей на базовых настройках показал сопоставимые значения accuracy у Логистической регрессии и Дерева решений на уровне ниже требуемого 0,71-0,73. 
Лучшие результаты у модели Случайный лес - 0,78-0,8, что соответствует требованиям задания.
2. Модель Логистическая регрессия в ходе данного задания не показала признаков переобучения. Модели Дерево решений и Случайный лес показали признак переобучения accuracy = 1.0
3. Целесообразно проведение обучения с настройкой гиперпараметров.

# Настройка гиперпараметров моделей

В связи с тем что значение accuracy на учебных выборках показывает значение свидетельствующее о переобучении целесообразно попробовать провести подстройку моделей за счет настройки гиперпараметров:

In [1]:
accuracy_regre_better = 0

for i in range (2000,11000,1000):
    model_regre1 = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=i)
    model_regre1.fit(train_data,train_target)
    accuracy_regre1 = model_regre1.score(valid_data,valid_target)
    if accuracy_regre1 > accuracy_regre:
        if accuracy_regre1 > accuracy_regre_better:
            accuracy_regre_better = accuracy_regre1
            iter_regre_better = i
            overeducate_better = model_regre1
            
if accuracy_regre_better > 0:
    print('Для модели логистической регрессии не достигнуто улучшение значения Accuracy на валидационной выборке')
    print('Оценка на валидации -', accuracy_regre_better, 'против -',accuracy_regre,'на базовых настройках')
    print('Оценка Accuracy на учебной выборке -', overeducate_better, 'против -',overeducate_regre_base, 'на базовых настройках')
else:
    print('Для модели логистической регрессии не достигнуто улучшение значения Accuracy на валидационной выборке')
        

NameError: name 'LogisticRegression' is not defined

In [15]:
accuracy_tree_better = 0
for i_tree_depth in range (1,5):
    for i_tree_leaf in range (1,100):
        model_tree1 = DecisionTreeClassifier(random_state=12345,max_depth = i_tree_depth,min_samples_leaf = i_tree_leaf)
        model_tree1.fit(train_data,train_target)
        accuracy_tree1 = model_tree1.score(valid_data,valid_target)
        if accuracy_tree1 > accuracy_tree:
            if accuracy_tree1 > accuracy_tree_better:
                accuracy_tree_better = accuracy_tree1
                iter_tree_better_depth = i_tree_depth
                iter_tree_better_leaf = i_tree_leaf 
                overeducate_tree_better = model_tree1.score(train_data,train_target)
                best_model_tree = model_tree1
if accuracy_tree_better > 0:
    print('Для модели Дерево решений достигнуто улучшение значения Accuracy на валидационной выборке')
    print('Оценка на валидации -', accuracy_tree_better, 'против -',accuracy_tree,'на базовых настройках')
    print('Оценка Accuracy на учебной выборке -', overeducate_tree_better, 'против -',overeducate_tree_base, 'на базовых настройках')
    print('Параметры модели с лучшими показателями:', best_model_tree)
else:
    print('Для модели Дерево решений не достигнуто улучшение значения Accuracy на валидационной выборке')
               

Для модели Дерево решений достигнуто улучшение значения Accuracy на валидационной выборке
Оценка на валидации - 0.7698289269051322 против - 0.7122861586314152 на базовых настройках
Оценка Accuracy на учебной выборке - 0.8184647302904564 против - 1.0 на базовых настройках
Параметры модели с лучшими показателями: DecisionTreeClassifier(max_depth=4, min_samples_leaf=8, random_state=12345)


In [16]:
accuracy_forest_better = 0
for i_forest_leaf in range (5,15):
    for i_forest_estimators in range (100,1201,100):
        model_forest1 = RandomForestClassifier(random_state=12345,min_samples_leaf = i_forest_leaf, n_estimators = i_forest_estimators)
        model_forest1.fit(train_data,train_target)
        accuracy_forest1 = model_forest1.score(valid_data,valid_target)
        if accuracy_forest1 > accuracy_forest:
            if accuracy_forest1 > accuracy_forest_better:
                accuracy_forest_better = accuracy_forest1
                iter_forest_better_leaf = i_forest_leaf
                iter_forest_better_estimators = i_forest_estimators
                overeducate_forest_better = model_forest1.score(train_data,train_target)
                best_model_forest = model_forest1
if accuracy_forest_better > 0:
    print('Для модели Случайный лес достигнуто улучшение значения Accuracy на валидационной выборке')
    print('Оценка на валидации -', accuracy_forest_better, 'против -',accuracy_forest,'на базовых настройках')
    print('Оценка Accuracy на учебной выборке -', overeducate_forest_better, 'против -',overeducate_forest_base, 'на базовых настройках')
    print('Параметры модели с лучшими показателями:', best_model_forest)
else:
    print('Для модели Случайный лес не достигнуто улучшение значения Accuracy на валидационной выборке')

Для модели Случайный лес достигнуто улучшение значения Accuracy на валидационной выборке
Оценка на валидации - 0.7978227060653188 против - 0.7947122861586314 на базовых настройках
Оценка Accuracy на учебной выборке - 0.8412863070539419 против - 1.0 на базовых настройках
Параметры модели с лучшими показателями: RandomForestClassifier(min_samples_leaf=13, n_estimators=200,
                       random_state=12345)


Для полученной модели значения min_samples_leaf и n_estimators находятся в пределах заданных диапазонов тестирования модели. Таким образом расширение диапазонов тестирования не целесообразно.

# Финальное тестирование и вывод

Лучшие значения Accuracy показала оптимизированная за счет настройки гиперпараметров модель Случайный лес:</p>

Accuracy на валидационной выборке - 0.7978, на учебной/тренировочной выборке - 0.8412.

Указанные значения соответствуют установленным требованиям задания, признаки переобучения у полученной модели не установлены.
Осуществим дополнительную оценку качества подготовленной модели Случайный лес на Тестовой выборке

In [17]:
print('Оценка Accuracy на тестовой выборке -', best_model_forest.score(test_data,test_target))

Оценка Accuracy на тестовой выборке - 0.7978227060653188


Таким образом сформирована модель соответствующая требованиям задания с учетом представленного заказчиком датасета.